<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/SQLAlchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import sqlite3
from tabulate import tabulate

db = 'chat.db'

def show_tables():
  tables = ['users', 'chats', 'messages']
  def show(tablename):
    try:
      with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(f'SELECT * FROM {tablename}')
        results = cursor.fetchall()
        # print(results)
        headers = []
        for i in range(len(cursor.description)):
          headers.append(cursor.description[i][0])
        print(tablename)
        print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
        print("\n")
    except:
      print("Empty")

  for table in tables:
    show(table)

show_tables()

users
+------+----------+
|   id |   name   |
+======+==========+
|    1 | John Doe |
+------+----------+


chats
+------+-----------+---------+----------------------------+
|   id |   user_id |  name   |         created_at         |
+======+===========+=========+============================+
|    1 |         1 | My Chat | 2023-11-13 00:50:33.675842 |
+------+-----------+---------+----------------------------+


messages
+------+-----------+-----------+---------------+----------------------------+
|   id |   user_id |   chat_id |    message    |         created_at         |
+======+===========+===========+===============+============================+
|    1 |         1 |         1 | Hello, world! | 2023-11-13 00:50:33.676459 |
+------+-----------+-----------+---------------+----------------------------+




In [104]:
import datetime

# Импортировать необходимые модули
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, Text, ForeignKey

# Создать engine для подключения к базе данных
engine = create_engine("sqlite:///chat.db")

# Создать метамодель для базы данных
metadata = MetaData()

# Создать таблицы базы данных
users_table = Table(
    "users",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(255)),
)
chats_table = Table(
    "chats",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("users.id")),
    Column("name", String(255)),
    Column("created_at", DateTime, default=datetime.datetime.utcnow()),
)
messages_table = Table(
    "messages",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("users.id")),
    Column("chat_id", Integer, ForeignKey("chats.id")),
    Column("message", Text),
    Column("created_at", DateTime, default=datetime.datetime.utcnow()),
)

# Добавить таблицы в базу данных
metadata.create_all(engine)

# Метод добавления пользователя
def add_user(name):
    # Создать пользователя
    user = {"name": name}

    # Добавить пользователя в базу данных
    with engine.connect() as connection:
        connection.execute(users_table.insert(), [user])
        connection.commit()

# Метод добавления нового чата по имени пользователя и названию чата с помощью join
def add_chat(user_name, chat_name):
    # Получить пользователя
    with engine.connect() as connection:
        user = connection.execute(
            users_table.select().where(users_table.c.name == user_name)
        ).fetchone()

    # Создать новый чат
    chat = {"name": chat_name, "user_id": user.id}

    # Добавить чат в базу данных
    with engine.connect() as connection:
        connection.execute(chats_table.insert(), [chat])
        connection.commit()

# Метод добавления нового сообщения по имени пользователя и названию чата с помощью join
def add_message(user_name, chat_name, message):
    # Получить пользователя
    with engine.connect() as connection:
        user = connection.execute(
            users_table.select().where(users_table.c.name == user_name)
        ).fetchone()

    # Получить чат
    with engine.connect() as connection:
        chat = connection.execute(
            chats_table.select().where(chats_table.c.name == chat_name)
        ).fetchone()

    # Создать новое сообщение
    message = {"chat_id": chat.id, "user_id": user.id, "message": message}

    # Добавить сообщение в базу данных
    with engine.connect() as connection:
        connection.execute(messages_table.insert(), [message])
        connection.commit()

# Пример использования методов
add_user("John Doe")
add_chat("John Doe", "My Chat")
add_message("John Doe", "My Chat", "Hello, world!")
